In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import time
from statsmodels.tsa.holtwinters import Holt
from statsmodels.tsa.arima_model import ARIMA
# Simple Exponential Smoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn import metrics
#pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)

I have uploaded the dataset in kaggle. You can access the dataset from here 

https://www.kaggle.com/krishnasrujan/frankfurt-stock-exchange


In [ ]:
#Frankfurt stock exchange
df=pd.read_csv('/kaggle/input/frankfurt-stock-exchange/frankfurt_stock_exchange.csv',parse_dates=['Date'])#,index_col='Date')
print(df.info())
df.head()

Dropping uneccessary columns

In [ ]:
df.drop(columns=['Change','Last Price of the Day','Daily Traded Units','Daily Turnover','Turnover','Traded Volume'],inplace=True)
df.head()

Filling null values by forward fill method

In [ ]:
data = df
for i in ['Open','Close','High','Low']:
    data[i] = data[i].ffill()
data

Creating separate columns for year,month and day. This helps us to check trend and seasonality in the data.

In [ ]:
data = data[169:]
data['year'] = pd.DatetimeIndex(data['Date']).year
data['month'] = [d.strftime('%b') for d in data.Date]
data['day'] = pd.DatetimeIndex(data['Date']).dayofweek
data.set_index('Date',inplace=True)

In [ ]:
data

If we see the trend and seasonality plots, there is some trend in the data but there is no seasonality
 either in weekdays or in months.

In [ ]:
# Draw Plot
fig, axes = plt.subplots(3, 1, figsize=(20,15), dpi= 80)
sns.boxplot(x='year', y='Close', data=data, ax=axes[0])
sns.boxplot(x='month', y='Close', data=data,ax = axes[1])
sns.boxplot(x='day', y='Close', data=data,ax = axes[2])

# Set Title
axes[0].set_title('Year-wise Box Plot\n(The Trend)', fontsize=18); 

axes[1].set_title('Month-wise Box Plot\n(The Seasonality)', fontsize=18)

axes[2].set_title('Day-wise Box Plot\n(The Seasonality)', fontsize=18)
plt.show()

Let's check the plot of closing price

In [ ]:
plt.plot(data['Close'])

Performing seasonal decompose. You can see the trend in the data. It is trying to find out seasonality but since there is no seasonality in the data, it is filled with blue color. Also notice that there is no pattern in the residue.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse

# Import Data
# Multiplicative Decomposition 
#result_mul = seasonal_decompose(data['Total_Sunspots'], model='multiplicative', extrapolate_trend='freq')

# Additive Decomposition
result_add = seasonal_decompose(data['Close'], model='add', extrapolate_trend='freq',freq=5)

# Plot
plt.rcParams.update({'figure.figsize': (5,5)})
#result_mul.plot().suptitle('Multiplicative Decompose', fontsize=22)
result_add.plot().suptitle('Additive Decompose', fontsize=22)
plt.show()

Detrending the closing price manually.

In [ ]:
data['rolling_mean'] = data['Close'].rolling(12).mean()
data['Detrend'] = data['Close'] - data['rolling_mean']
plt.plot(data['rolling_mean'])
plt.title('Trend',fontsize=16)

Deseasonalizing the data with seasonal decompose. Since it doesn't have any seasonality, plot doesn't change much.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result_add = seasonal_decompose(data['Close'], model='mul', extrapolate_trend='freq',freq=96)
deseasonalized = data.Close.values -result_add.seasonal
plt.plot(deseasonalized)
plt.title('Drug Sales deseasonalized by subtracting the seasonal component', fontsize=16)

Pandas autocorrelation plot helps to find the dependency of current value on previous values. As there are no patterns in the below plot we cannot get any intuition.

In [ ]:
from pandas.plotting import autocorrelation_plot


# Draw Plot
plt.rcParams.update({'figure.figsize':(9,5), 'figure.dpi':120})
autocorrelation_plot(data['Close'].tolist())

Below plots of autocorrelation and partial autocorrelation play important role in determing the p and q values for ARIMA.

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Calculate ACF and PACF upto 50 lags
# acf_50 = acf(df.value, nlags=50)
# pacf_50 = pacf(df.value, nlags=50)

# Draw Plot
fig, axes = plt.subplots(1,2,figsize=(16,3), dpi= 100)
plot_acf(data['Close'].tolist(), lags=200, ax=axes[0])
plot_pacf(data['Close'].tolist(), lags=50, ax=axes[1])

## Holt Winter's

Holt Winters model doesn't work well on the data. It basically predicts according the the trend. 

In [ ]:
count = int(data.shape[0]*0.8)
Train = data[:count]
Test = data[count:]

y_hat_avg = Test.copy()
fit1 = Holt(np.asarray(Train['Close'])).fit()
y_hat_avg['Holt_Winter'] = fit1.predict(start=count+1,end=data.shape[0])
plt.figure(figsize=(16,8))
plt.plot(Train.index, Train['Close'], label='Train')
plt.plot(Test.index,Test['Close'], label='Test')
plt.plot(y_hat_avg.index,y_hat_avg['Holt_Winter'], label='Holt_Winter')
plt.legend(loc='best')

## ARIMA model

In [ ]:
new = data[['Close']]

In [ ]:
Train, Test = new.iloc[:count,0], new.iloc[count:,0]
history = [x for x in Train]

We make a list for training data and we will be predicting next day's closing price with this data. Later the predicted value is appended to training data and next day's value is predicted again. This is repeated for all the test data.


ARIMA's forecast function gives 4 outputs.

predicted value

standard error

lower and upper confidance values which are very important when predicting on time series data.


In [ ]:
predictions = []
lower_list = []
upper_list = []
for t in range(len(Test)):
    model = ARIMA(history, order=(2,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    lower = output[2][0][0]
    upper = output[2][0][1]
    predictions.append(yhat)
    lower_list.append(lower)
    upper_list.append(upper)
    obs = Test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

In [ ]:
error = metrics.mean_squared_error(Test, predictions)
print('Test MSE: %.3f' % error)

Below is the plot for orginal test data values, predicted values and confidance interval.

You can see that the predicted values and original values are collinear. 

In [ ]:
# plot
plt.plot(Test.values,color='black')
plt.plot(lower_list,color='red')
plt.plot(upper_list,color='green')
plt.plot(predictions)
plt.show()

## Prophet

In [ ]:
new = data[['Close']]
new.reset_index(inplace=True)
new.columns = ['ds','y']
new

In [ ]:
new.shape

In [ ]:
from fbprophet import Prophet
m = Prophet()
model = m.fit(new)

In [ ]:
future = m.make_future_dataframe(periods=30)
future.tail()

In [ ]:
future.shape, new.shape

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast['yhat'][:2800].shape
new['y'].shape

In [ ]:
fig1 = m.plot(forecast)

## Random Forest Regressor

In [ ]:
data['HL_PCT']=(data['High']-data['Close'])/data['Close']*100
data['PCT_change']=(data['Close']-data['Open'])/data['Open']*100
data.drop(['year','day','rolling_mean','Detrend','Open','High','Low','month'],axis=1,inplace=True)
data

In [ ]:
forecast_out=4
data['forecast_col'] = data['Close'].shift(-forecast_out)
data.dropna(inplace=True)
data

In [ ]:
train_count =int(data.shape[0]*0.85)
test_count = data.shape[0]-train_count

In [ ]:
y_train = data.iloc[:train_count,3]
y_test  = data.iloc[train_count:,3]
print(y_train.shape,y_test.shape)

In [ ]:
x_train = data.iloc[:train_count,0:-1]
x_test = data.iloc[train_count:,0:-1]

In [ ]:
x_lately=x_train[-forecast_out:]
x_train=x_train[:-forecast_out]
df.dropna(inplace=True)
y_train=y_train[:-forecast_out]

In [ ]:
print(y_train.shape)

In [ ]:
x_train

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor()
clf.fit(x_train,y_train)
y_preds = clf.predict(x_test)
y_lately = clf.predict(x_lately)

In [ ]:
plt.plot(y_preds,color='red')
plt.plot(y_test)
plt.show()

In [ ]:
from sklearn import metrics
print(metrics.mean_squared_error(y_preds,y_test))

## Ramdom Forest with parameter tuning

In [ ]:
clf = RandomForestRegressor(n_estimators = 100,min_samples_leaf = 3,min_samples_split = 8)
clf.fit(x_train,y_train)
y_preds = clf.predict(x_test)
y_lately = clf.predict(x_lately)

In [ ]:
plt.plot(y_preds,color='red')
plt.plot(y_test)
plt.show()

In [ ]:
from sklearn import metrics
print(metrics.mean_squared_error(y_preds,y_test))